In [1]:
! wget -P ../../data http://files.grouplens.org/datasets/movielens/ml-latest-small.zip

--2020-11-07 17:47:00--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘../../data/ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.006s  

2020-11-07 17:47:01 (153 MB/s) - ‘../../data/ml-latest-small.zip’ saved [978202/978202]



Import Zipfile and unzip the small dataset

In [111]:
import zipfile
with zipfile.ZipFile('../../data/ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('../../../data')

In [112]:
from random import gauss as gs, uniform as uni, seed
import numpy as np
import pandas as pd

Import CSVs and save them as dataframes

In [113]:
links_df = pd.read_csv('../../../data/ml-latest-small/links.csv')
movies_df = pd.read_csv('../../../data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('../../../data/ml-latest-small/ratings.csv')
tags_df = pd.read_csv('../../../data/ml-latest-small/tags.csv')

In [114]:
len(links_df)

9742

In [115]:
len(movies_df)

9742

In [116]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [117]:
tags_df

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [118]:
movie_ratings = links_df.merge(movies_df, on=["movieId"])
movie_ratings.head()

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy
2,3,113228,15602.0,Grumpier Old Men (1995),Comedy|Romance
3,4,114885,31357.0,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,113041,11862.0,Father of the Bride Part II (1995),Comedy


In [119]:
movie_ratings = ratings_df.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId,movieId,rating,timestamp,imdbId,tmdbId,title,genres
0,1,1,4.0,964982703,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [120]:
len(movie_ratings)

100836

In [121]:
movie_ratings = tags_df.merge(movie_ratings, on=["movieId"])
movie_ratings.head()

,userId_x,movieId,tag,timestamp_x,userId_y,rating,timestamp_y,imdbId,tmdbId,title,genres
0,2,60756,funny,1445714994,2,5.0,1445714980,838283,12133.0,Step Brothers (2008),Comedy
1,2,60756,funny,1445714994,18,3.0,1455749449,838283,12133.0,Step Brothers (2008),Comedy
2,2,60756,funny,1445714994,62,3.5,1528934376,838283,12133.0,Step Brothers (2008),Comedy
3,2,60756,funny,1445714994,68,2.5,1269123243,838283,12133.0,Step Brothers (2008),Comedy
4,2,60756,funny,1445714994,73,4.5,1464196221,838283,12133.0,Step Brothers (2008),Comedy


### FSM

In [122]:
# import necessary libraries
from pyspark.sql import SparkSession

# instantiate SparkSession object
# spark = SparkSession.builder.master('local').getOrCreate()

spark = SparkSession\
        .builder\
        .appName('ALSExample').config('spark.driver.host', 'localhost')\
        .getOrCreate()

# read in the dataset into pyspark DataFrame
links_pdf = spark.read.csv('../../../data/ml-latest-small/links.csv', header='true', inferSchema='true')
movies_pdf = spark.read.csv('../../../data/ml-latest-small/movies.csv', header='true', inferSchema='true')
ratings_pdf = spark.read.csv('../../../data/ml-latest-small/ratings.csv', header='true', inferSchema='true')
tags_pdf = spark.read.csv('../../../data/ml-latest-small/tags.csv', header='true', inferSchema='true')

In [124]:
movie_ratings = links_pdf.merge(movies_pdf, on=["movieId"])
movie_ratings = ratings_pdf.merge(movie_ratings, on=["movieId"])
movie_ratings = tags_pdf.merge(movie_ratings, on=["movieId"])

AttributeError: 'DataFrame' object has no attribute 'merge'

In [125]:
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

# split into training and testing sets
(training, test) = ratings_pdf.randomSplit([0.8, 0.2])

# Build the recommendation model using ALS on the training data
# Note we set cold start strategy to 'drop' to ensure we don't get NaN evaluation metrics
als = ALS(maxIter=5,rank=4, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating',
          coldStartStrategy='drop')

# fit the ALS model to the training set
model = als.fit(training)


In [126]:
# importing appropriate library
from pyspark.ml.evaluation import RegressionEvaluator

# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))

Root-mean-square error = 0.9818640252458379


### Cross Validating and Param Testing

In [128]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# initialize the ALS model
als_model = ALS(userCol='userId', itemCol='movieId', 
                ratingCol='rating', coldStartStrategy='drop')

# create the parameter grid                 
params = ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001, 0.1])\
          .addGrid(als_model.rank, [4, 10, 50]).build()


# instantiating crossvalidator estimator
cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(ratings_pdf)    

# We see the best model has a rank of 50, so we will use that in our future models with this dataset
best_model.bestModel.rank

50

In [136]:
best_model.bestModel.regParam

AttributeError: 'ALSModel' object has no attribute 'regParam'

In [134]:
print("\n\n  ParamMap:", best_model._java_obj.parent().extractParamMap())

AttributeError: 'CrossValidatorModel' object has no attribute '_java_obj'